<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-01-30 03:05:56--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.1’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.03s   

2020-01-30 03:05:56 (612 KB/s) - ‘house-votes-84.data.1’ saved [18171/18171]



In [3]:
column_headers = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']

df = pd.read_csv('house-votes-84.data',
                 header=None,
                 names=column_headers,
                 na_values='?')

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [4]:
df = df.replace({'y':1, 'n':0})
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [5]:
republicans = df[df['party'] == 'republican']
republicans.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [6]:
democrats = df[df['party'] == 'democrat']
democrats.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [0]:
party_support = df.groupby('party').mean()

In [8]:
party_support.head()

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
party,,,,,,,,,,,,,,,,
democrat,0.604651,0.502092,0.888462,0.054054,0.215686,0.476744,0.772201,0.828897,0.758065,0.471483,0.505882,0.144578,0.289683,0.350195,0.637450,0.935135
republican,0.187879,0.506757,0.134146,0.987879,0.951515,0.897590,0.240741,0.152866,0.115152,0.557576,0.132075,0.870968,0.860759,0.981366,0.089744,0.657534


In [9]:
x = []
for each in party_support:
    if party_support[each][0] > party_support[each][1]:
        x.append('democrats')
    else:
        x.append('republicans')

print(x)

['democrats', 'republicans', 'democrats', 'republicans', 'republicans', 'republicans', 'democrats', 'democrats', 'democrats', 'republicans', 'democrats', 'republicans', 'republicans', 'republicans', 'democrats', 'democrats']


In [0]:
new = pd.DataFrame(x).T

In [11]:
new.columns = ['handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']
new.head()

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,democrats,republicans,democrats,republicans,republicans,republicans,democrats,democrats,democrats,republicans,democrats,republicans,republicans,republicans,democrats,democrats


In [0]:
party_support = party_support.append(new)

In [0]:
party_support = party_support.rename(index={0:'supports_more'})

In [14]:
x = []
for each in party_support:
    p = '{:15f}'.format(ttest_ind(democrats[each], republicans[each], nan_policy='omit').pvalue)
    x.append(p)
print(x)


['       0.000000', '       0.929156', '       0.000000', '       0.000000', '       0.000000', '       0.000000', '       0.000000', '       0.000000', '       0.000000', '       0.083302', '       0.000000', '       0.000000', '       0.000000', '       0.000000', '       0.000000', '       0.000000']


In [15]:
new2 = pd.DataFrame(x).T
new2.columns = ['handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']

new2.head()

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,0.000000,0.929156,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083302,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
party_support = party_support.append(new2)
party_support.head()

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
democrat,0.604651,0.502092,0.888462,0.0540541,0.215686,0.476744,0.772201,0.828897,0.758065,0.471483,0.505882,0.144578,0.289683,0.350195,0.63745,0.935135
republican,0.187879,0.506757,0.134146,0.987879,0.951515,0.89759,0.240741,0.152866,0.115152,0.557576,0.132075,0.870968,0.860759,0.981366,0.0897436,0.657534
supports_more,democrats,republicans,democrats,republicans,republicans,republicans,democrats,democrats,democrats,republicans,democrats,republicans,republicans,republicans,democrats,democrats
0,0.000000,0.929156,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083302,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
party_support = party_support.rename(index={0:'p_value'})
party_support.head()

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
democrat,0.604651,0.502092,0.888462,0.0540541,0.215686,0.476744,0.772201,0.828897,0.758065,0.471483,0.505882,0.144578,0.289683,0.350195,0.63745,0.935135
republican,0.187879,0.506757,0.134146,0.987879,0.951515,0.89759,0.240741,0.152866,0.115152,0.557576,0.132075,0.870968,0.860759,0.981366,0.0897436,0.657534
supports_more,democrats,republicans,democrats,republicans,republicans,republicans,democrats,democrats,democrats,republicans,democrats,republicans,republicans,republicans,democrats,democrats
p_value,0.000000,0.929156,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083302,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [22]:
for each in party_support:
    if float(party_support[each]['p_value']) < .01:
        print('At the desired confidence level we reject the hypothesis that there is not a difference in how democrats and republicans support: ' + each + ', and it is more supported by ' + party_support[each]['supports_more'])
    else:
        print('We fail to reject the hypothesis that there is a difference in support between democrats and republicans on the issue of: ' + each)
   

At the desired confidence level we reject the hypothesis that there is not a difference in how democrats and republicans support: handicapped-infants, and it is more supported by democrats
We fail to reject the hypothesis that there is a difference in support between democrats and republicans on the issue of: water-project
At the desired confidence level we reject the hypothesis that there is not a difference in how democrats and republicans support: budget, and it is more supported by democrats
At the desired confidence level we reject the hypothesis that there is not a difference in how democrats and republicans support: physician-fee-freeze, and it is more supported by republicans
At the desired confidence level we reject the hypothesis that there is not a difference in how democrats and republicans support: el-salvador-aid, and it is more supported by republicans
At the desired confidence level we reject the hypothesis that there is not a difference in how democrats and republicans

In [0]:
#writing a function 
def can_we_reject_the_null(topic, critical_p_value):
    '''The topic has to be a string.'''
    for each in party_support:
        if (str(topic) == each) and (float(party_support[each]['p_value']) < critical_p_value):
            print('At the desired confidence level we reject the hypothesis that there is not a difference in how democrats and republicans support: ' + each + ', and it is more supported by ' + party_support[each]['supports_more'])
        elif (str(topic) == each):
            print('We fail to reject the null hypothesis')
            
    

In [32]:
can_we_reject_the_null('water-project', .01)

We fail to reject the null hypothesis


In [33]:
can_we_reject_the_null('handicapped-infants', .01)

At the desired confidence level we reject the hypothesis that there is not a difference in how democrats and republicans support: handicapped-infants, and it is more supported by democrats


In [34]:
can_we_reject_the_null('physician-fee-freeze', .01)

At the desired confidence level we reject the hypothesis that there is not a difference in how democrats and republicans support: physician-fee-freeze, and it is more supported by republicans
